# Introduction to `cf_xarray`

This notebook is a brief introduction to `cf_xarray`'s current capabilities.


In [ ]:
import cf_xarray as cfxr
import numpy as np
import xarray as xr

Lets read two datasets.


In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
ds.air.attrs["standard_name"] = "air_temperature"
ds

This one is inspired by POP model output and illustrates how the coordinates
attribute is interpreted. It also illustrates one way of tagging curvilinear
grids for convenient use of `cf_xarray`


In [ ]:
from cf_xarray.datasets import popds as pop

pop

This synthetic dataset has multiple `X` and `Y` coords. An example would be
model output on a staggered grid.


In [ ]:
from cf_xarray.datasets import multiple

multiple

This dataset has ancillary variables


In [ ]:
from cf_xarray.datasets import anc

anc

## What attributes have been discovered?

The criteria for identifying variables using CF attributes are listed
[here](../criteria.rst).


In [ ]:
ds.lon

`ds.lon` has attributes `axis: X`. This means that `cf_xarray` can identify the
`'X'` axis as being represented by the `lon` variable.

It can also use the `standard_name` and `units` attributes to infer that `lon`
is "Longitude". To see variable names that `cf_xarray` can infer, use `ds.cf`


In [ ]:
ds.cf

For `pop`, only `latitude` and `longitude` are detected, not `X` or `Y`. Please
comment here: https://github.com/xarray-contrib/cf-xarray/issues/23 if you have
opinions about this behaviour.


In [ ]:
pop.cf

For `multiple`, multiple `X` and `Y` coordinates are detected


In [ ]:
multiple.cf

## Feature: Accessing coordinate variables

`.cf` implements `__getitem__` to allow easy access to coordinate and axis
variables.


In [ ]:
ds.cf["X"]

Indexing with a scalar key raises an error if the key maps to multiple variables
names


In [ ]:
multiple.cf["X"]

In [ ]:
pop.cf["longitude"]

To get back all variables associated with that key, pass a single element list
instead.


In [ ]:
multiple.cf[["X"]]

In [ ]:
pop.cf[["longitude"]]

DataArrays return DataArrays


In [ ]:
pop.UVEL.cf["longitude"]

`Dataset.cf[...]` returns a single `DataArray`, parsing the `coordinates`
attribute if present, so we correctly get the `TLONG` variable and not the
`ULONG` variable


In [ ]:
pop.cf["TEMP"]

`Dataset.cf[...]` also interprets the `ancillary_variables` attribute. The
ancillary variables are returned as coordinates of a DataArray


In [ ]:
anc.cf["q"]

## Feature: Accessing variables by standard names


In [ ]:
pop.cf[["sea_water_potential_temperature", "UVEL"]]

Note that ancillary variables are included as coordinate variables


In [ ]:
anc.cf["specific_humidity"]

## Feature: Utility functions

There are some utility functions to allow use by downstream libraries


In [ ]:
pop.cf.keys()

You can test for presence of these keys


In [ ]:
"sea_water_x_velocity" in pop.cf

You can also get out the available Axis names


In [ ]:
pop.cf.axes

or available Coordinate names. Same for cell measures (`.cf.cell_measures`) and
standard names (`.cf.standard_names`).


In [ ]:
pop.cf.coordinates

**Note:** Although it is possible to assign additional coordinates,
`.cf.coordinates` only returns a subset of
`("longitude", "latitude", "vertical", "time")`.


## Feature: Rewriting property dictionaries

`cf_xarray` will rewrite the `.sizes` and `.chunks` dictionaries so that one can
index by a special CF axis or coordinate name


In [ ]:
ds.cf.sizes

Note the duplicate entries above:

1. One for `X`, `Y`, `T`
2. and one for `longitude`, `latitude` and `time`.

An error is raised if there are multiple `'X'` variables (for example)


In [ ]:
multiple.cf.sizes

In [ ]:
multiple.v1.cf.sizes

## Feature: Renaming coordinate variables

`cf_xarray` lets you rewrite coordinate variables in one dataset to like
variables in another dataset. This can only be done when a one-to-one mapping is
possible

In this example, `TLONG` and `TLAT` are renamed to `lon` and `lat` i.e. their
counterparts in `ds`. Note the the `coordinates` attribute is appropriately
changed.


In [ ]:
pop.cf["TEMP"].cf.rename_like(ds)

## Feature: Rewriting arguments

`cf_xarray` can rewrite arguments for a large number of xarray functions. By
this I mean that instead of specifing say `dim="lon"`, you can pass `dim="X"` or
`dim="longitude"` and `cf_xarray` will rewrite that to `dim="lon"` based on the
attributes present in the dataset.

Here are a few examples


### Slicing


In [ ]:
ds.air.cf.isel(T=1)

Slicing works will expand a single key like `X` to multiple dimensions if those
dimensions are tagged with `axis: X`


In [ ]:
multiple.cf.isel(X=1, Y=1)

### Reductions


In [ ]:
ds.air.cf.mean("X")

Expanding to multiple dimensions is also supported


In [ ]:
# takes the mean along ["x1", "x2"]
multiple.cf.mean("X")

### Plotting


In [ ]:
ds.air.cf.isel(time=1).cf.plot(x="X", y="Y")

In [ ]:
ds.air.cf.isel(T=1, Y=[0, 1, 2]).cf.plot(x="longitude", hue="latitude")

`cf_xarray` can facet


In [ ]:
seasonal = (
    ds.air.groupby("time.season")
    .mean()
    .reindex(season=["DJF", "MAM", "JJA", "SON"])
)
seasonal.cf.plot(x="longitude", y="latitude", col="season")

### Resample & groupby


In [ ]:
ds.cf.resample(T="D").mean()

`cf_xarray` also understands the "datetime accessor" syntax for groupby


In [ ]:
ds.cf.groupby("T.month").mean("longitude")

### Rolling & coarsen


In [ ]:
ds.cf.rolling(X=5).mean()

`coarsen` works but everything later will break because of xarray bug
https://github.com/pydata/xarray/issues/4120

`ds.isel(lon=slice(50)).cf.coarsen(Y=5, X=10).mean()`


## Feature: mix "special names" and variable names


In [ ]:
ds.cf.groupby("T.month").mean(["lat", "X"])

## Feature: Weight by Cell Measures

`cf_xarray` can weight by cell measure variables if the appropriate attribute is
set


In [ ]:
# Lets make some weights (not sure if this is right)
ds.coords["cell_area"] = (
    np.cos(ds.air.cf["latitude"] * np.pi / 180)
    * xr.ones_like(ds.air.cf["longitude"])
    * 105e3
    * 110e3
)
# and set proper attributes
ds.air.attrs["cell_measures"] = "area: cell_area"

In [ ]:
ds.air.cf.weighted("area").mean(["latitude", "time"]).cf.plot(x="longitude")
ds.air.mean(["lat", "time"]).cf.plot(x="longitude")

## Feature: Cell boundaries and vertices

`cf_xarray` can infer cell boundaries (for rectilinear grids) and convert
CF-standard bounds variables to vertices.


In [ ]:
ds_bnds = ds.cf.add_bounds(["lat", "lon"])
ds_bnds

We can also convert each bounds variable independently with the top-level
functions


In [ ]:
lat_bounds = ds_bnds.cf.get_bounds("latitude")

lat_vertices = cfxr.bounds_to_vertices(lat_bounds, bounds_dim="bounds")
lat_vertices

In [ ]:
# Or we can convert _all_ bounds variables on a dataset
ds_crns = ds_bnds.cf.bounds_to_vertices()
ds_crns